In [2]:
import os
import requests
from dotenv import load_dotenv
import pandas as pd

### Step 1: Fetch Movie Data from API


In [3]:
load_dotenv(".env")
api_key = os.getenv("API_KEY")
if not api_key:
    raise ValueError("API_KEY not found.") 
movie_ids = [
    0, 299534, 19995, 140607, 299536, 597, 135397, 420818,
    24428, 168259, 99861, 284054, 12445, 181808, 330457,
    351286, 109445, 321612, 260513
]
selected_movie_ids = []
for movie_id in movie_ids:
    if movie_id == 0:
        continue
    url = f"https://api.themoviedb.org/3/movie/{movie_id}?append_to_response=credits&api_key={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        selected_movie_ids.append(response.json())
    else:
        print(f"Failed to fetch movie {movie_id}: {response.status_code}")

# Store data as a Pandas DataFrame

In [4]:
df_movie = pd.DataFrame(selected_movie_ids) #Converted DataFrame

In [5]:
df_movie['id'] # check if data is loaded correctly

0     299534
1      19995
2     140607
3     299536
4        597
5     135397
6     420818
7      24428
8     168259
9      99861
10    284054
11     12445
12    181808
13    330457
14    351286
15    109445
16    321612
17    260513
Name: id, dtype: int64

In [6]:
df_movie.columns # check column names

Index(['adult', 'backdrop_path', 'belongs_to_collection', 'budget', 'genres',
       'homepage', 'id', 'imdb_id', 'origin_country', 'original_language',
       'original_title', 'overview', 'popularity', 'poster_path',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'video', 'vote_average', 'vote_count', 'credits'],
      dtype='object')

### End of Step 1: Fetch Movie Data from API

# Step 2: Data Cleaning and Preprocessing

In [ ]:
# 1: the irrelevant columns to drop
cols_to_drop = ['adult', 'imdb_id', 'original_title', 'video', 'homepage']
for col in cols_to_drop:
    # Check if the column actually exists in the DataFrame to avoid errors
    if col in cols_to_drop:
        df_movie.drop(col, axis=1, inplace=True)    # axis=1 refers to columns. inplace=True modifies the DataFrame directly.


In [15]:
df_movie.columns # check column names again after dropping columns

Index(['backdrop_path', 'belongs_to_collection', 'budget', 'genres', 'id',
       'origin_country', 'original_language', 'overview', 'popularity',
       'poster_path', 'production_companies', 'production_countries',
       'release_date', 'revenue', 'runtime', 'spoken_languages', 'status',
       'tagline', 'title', 'vote_average', 'vote_count', 'credits'],
      dtype='object')

In [59]:

# 1. Load data into DataFrame
df_movie = pd.DataFrame(selected_movie_ids)

# 2. Evaluate JSON-like columns
extracted_columns = ['belongs_to_collection', 'genres', 'spoken_languages', 'production_countries', 'production_companies']

# 3. Extract and clean points
for column in extracted_columns:
    if column == 'belongs_to_collection':
        # Extracts 'name' from belongs_to_collection dict, else None
        df_movie[column] = df_movie[column].apply(lambda x: x['name'] if isinstance(x, dict) else None)
    else:
        # Extracts other columns as pipe-separated strings
        df_movie[column] = df_movie[column].apply(lambda x: "|".join([item['name'] for item in x]) if isinstance(x, list) else None)

# 4. Display value counts for each extracted column
for col in extracted_columns:        
    print(f"Value counts for {col}:")
    print(df_movie[col].value_counts().head()) 
    print("\n")

Value counts for belongs_to_collection:
belongs_to_collection
The Avengers Collection     4
Star Wars Collection        2
Frozen Collection           2
Jurassic Park Collection    2
Avatar Collection           1
Name: count, dtype: int64


Value counts for genres:
genres
Adventure|Action|Science Fiction             3
Action|Adventure|Science Fiction|Thriller    2
Action|Adventure|Science Fiction             2
Action|Adventure|Fantasy|Science Fiction     1
Drama|Romance                                1
Name: count, dtype: int64


Value counts for spoken_languages:
spoken_languages
English                                              9
English|日本語|                                         1
English|Español                                      1
English|                                             1
English|Français|Deutsch|svenska|Italiano|Pусский    1
Name: count, dtype: int64


Value counts for production_countries:
production_countries
United States of America                   16
Unit

# Handling Missing & Incorrect Data

In [ ]:
# Convert column datatypes